In [19]:
### Find Missing Plants

In [1]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
from pudl import ferc1, pudl, models, models_ferc1, settings, constants, eia923, models_eia923
from pudl.eia923 import get_eia923_page
from pudl.eia923 import get_eia923_xlsx, get_eia923_column_map
from pudl.eia923 import get_eia923_plant_info
from pudl.pudl import *
from pudl.constants import tab_map_eia923
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from sqlalchemy import Integer, String, Numeric, Boolean, Float

In [2]:
%load_ext autoreload
%autoreload 1
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.eia923
%aimport pudl.constants
%aimport pudl.settings
%aimport pudl.models
%aimport pudl.models_eia923

In [3]:
eia923_years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [4]:
eia923_xlsx = get_eia923_xlsx(eia923_years)

Reading EIA 923 spreadsheet data for 2009.
Reading EIA 923 spreadsheet data for 2010.
Reading EIA 923 spreadsheet data for 2011.
Reading EIA 923 spreadsheet data for 2012.
Reading EIA 923 spreadsheet data for 2013.
Reading EIA 923 spreadsheet data for 2014.
Reading EIA 923 spreadsheet data for 2015.
Reading EIA 923 spreadsheet data for 2016.


In [5]:
eia923_plants = get_eia923_plant_info(eia923_years, eia923_xlsx)
eia923_plant_ids = eia923_plants[['plant_id',]]

Converting EIA 923 plant_frame to DataFrame...
Converting EIA 923 generation_fuel to DataFrame...
Converting EIA 923 boiler_fuel to DataFrame...
Converting EIA 923 generator to DataFrame...
Converting EIA 923 fuel_receipts_costs to DataFrame...


In [9]:
output_tab_plants = pd.read_excel('../id_mapping/mapping_eia923_ferc1.xlsx', 'plants_output', na_values='', keep_default_na=False,
                              converters={'plant_id': int,
                                          'plant_name': str,
                                          'respondent_id_ferc1': int,
                                          'respondent_name_ferc1': str,
                                          'plant_name_ferc1': str,
                                          'plant_id_eia923': int,
                                          'plant_name_eia923': str,
                                          'operator_name_eia923': str,
                                          'operator_id_eia923': int})
output_plant_ids = output_tab_plants[['plant_id_eia923']]
output_plant_ids = output_plant_ids.dropna()

In [18]:
eia923_plant_ids_tuple = set([ tuple(line) for line in eia923_plant_ids.values.tolist()])
output_plant_ids_tuple = set([ tuple(line) for line in output_plant_ids.values.tolist()])
missing_plant_ids = pd.DataFrame(list(eia923_plant_ids_tuple.difference(output_plant_ids_tuple)))
missing_plant_ids = missing_plant_ids.rename(columns={0: 'plant_id',})

In [12]:
missing_plants = missing_plant_ids.merge(eia923_plants[['plant_id','plant_name','operator_name','operator_id']],  on='plant_id', how='left')

In [17]:
missing_plants.to_csv('../id_mapping/missing_plants.csv', index=False)